In [8]:
# -*- coding = utf-8 -*-
"""
Main function to build recommendation systems.
"""
import utils
from ItemCF import ItemBasedCF
from LFM import LFM
from BasicSVD import BasicSVD
from UserCF import UserBasedCF
from dataset import DataSet
from most_popular import MostPopular
from random_pred import RandomPredict
from utils import LogTime


def run_model(model_name, dataset_name, test_size=0.3, clean=False, user_list = [1, 100, 233, 666, 888]):
    print('*' * 70)
    print('\tThis is %s model trained on %s with test_size = %.2f' % (model_name, dataset_name, test_size))
    print('*' * 70 + '\n')
    model_manager = utils.ModelManager(dataset_name, test_size)
    try:
        trainset = model_manager.load_model('trainset')
        testset = model_manager.load_model('testset')
    except OSError:
        ratings = DataSet.load_dataset(name=dataset_name)
        trainset, testset = DataSet.train_test_split(ratings, test_size=test_size)
        model_manager.save_model(trainset, 'trainset')
        model_manager.save_model(testset, 'testset')
    '''Do you want to clean workspace and retrain model again?'''
    '''if you want to change test_size or retrain model, please set clean_workspace True'''
    model_manager.clean_workspace(clean)
    if model_name == 'UserCF':
        model = UserBasedCF()
    elif model_name == 'ItemCF':
        model = ItemBasedCF()
    elif model_name == 'Random':
        model = RandomPredict()
    elif model_name == 'MostPopular':
        model = MostPopular()
    elif model_name == 'UserCF-IIF':
        model = UserBasedCF(use_iif_similarity=True)
    elif model_name == 'ItemCF-IUF':
        model = ItemBasedCF(use_iuf_similarity=True)
    elif model_name == 'LFM':
        # K, epochs, alpha, lamb, n_rec_movie
        model = LFM(10, 20, 0.1, 0.01, 10)
    elif model_name == 'BasicSVD':
        # K, epochs, alpha, lamb, n_rec_movie
        model = BasicSVD(10, 20, 0.1, 0.01, 10)
    else:
        raise ValueError('No model named ' + model_name)
    model.fit(trainset)
    model.test(testset)
    rec_result = recommend_test(model, user_list)
    return rec_result



def recommend_test(model, user_list):
    rec_result = {}
    for user in user_list:
        recommend = model.recommend(str(user))
        print("recommend for userid = %s:" % user)
        print(recommend)
        rec_result[user] = recommend
    return rec_result
main_time = LogTime(words="Main Function")
dataset_name = 'ml-100k'

model_type = 'LFM'
test_size = 0.1
# 用户id
user_id_list = [6]
rec_result = run_model(model_type, dataset_name, test_size, False, user_id_list)

import json
file_path = "data/user_movie_feature/movies.csv"  # 请替换为实际的文件路径
dic_movie = {} # 用于存储解析后的数据
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # 解析JSON对象
        record = json.loads(line.strip())
#         data.append(record)
#         print(record)
        dic_movie[str(record['id'])] = record['title']

rec_movie_name = [dic_movie[x] for x in rec_result[user_id_list[0]]]
print(rec_movie_name)

import pickle
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)
dic = {}
for name in rec_movie_name:
    if name in movie_data:
        print("find", name)
        dic[name] = movie_data[name]
        with open('show_movie_url_pic.pickle', 'wb') as f:
            pickle.dump(dic, f)
    else:
        print("not find", name)
with open('show_movie_url_pic.pickle', 'rb') as f:
        show_movie_data = pickle.load(f)
print(show_movie_data)


**********************************************************************
	This is LFM model trained on ml-100k with test_size = 0.10
**********************************************************************

LFM start...

User origin similarity model has saved before.
Load model success...

Test recommendation system start...
 steps(0), 0.00 seconds have spent..
Test recommendation system success.
total  step number is 943
total 1.99 seconds have spent

precision=0.2038	recall=0.1915	coverage=0.3010	popularity=5.2464

recommend for userid = 6:
['479', '181', '603', '172', '515', '435', '654', '185', '478', '705']
['Vertigo (1958)', 'Return of the Jedi (1983)', 'Rear Window (1954)', 'Empire Strikes Back, The (1980)', 'Boot, Das (1981)', 'Butch Cassidy and the Sundance Kid (1969)', 'Chinatown (1974)', 'Psycho (1960)', 'Philadelphia Story, The (1940)', "Singin' in the Rain (1952)"]
find Vertigo (1958)
find Return of the Jedi (1983)
find Rear Window (1954)
find Empire Strikes Back, The (1980)
f

In [ ]:

!python3 main2.py

 * Serving Flask app 'main2' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 514-957-196
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/Vertigo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/ReturnoftheJedi.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/ButchCassidyandtheSundanceKid.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/RearWindow.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/BootDas.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/Chinatown.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2024 09:13:06] "GET /static/pictures/EmpireStrikesBackThe.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20

In [4]:
import pickle
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [5]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [5]:
import json
file_path = "data/user_movie_feature/movies.csv"  # 请替换为实际的文件路径
dic_movie = {} # 用于存储解析后的数据
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # 解析JSON对象
        record = json.loads(line.strip())
#         data.append(record)
#         print(record)
        dic_movie[str(record['id'])] = record['title']

In [9]:
import pickle
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [10]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [8]:
dic_movie

{'4': 'Get Shorty (1995)',
 '5': 'Copycat (1995)',
 '6': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7': 'Twelve Monkeys (1995)',
 '8': 'Babe (1995)',
 '9': 'Dead Man Walking (1995)',
 '10': 'Richard III (1995)',
 '11': 'Seven (Se7en) (1995)',
 '12': 'Usual Suspects, The (1995)',
 '13': 'Mighty Aphrodite (1995)',
 '14': 'Postino, Il (1994)',
 '15': "Mr. Holland's Opus (1995)",
 '16': 'French Twist (Gazon maudit) (1995)',
 '17': 'From Dusk Till Dawn (1996)',
 '18': 'White Balloon, The (1995)',
 '19': "Antonia's Line (1995)",
 '20': 'Angels and Insects (1995)',
 '21': 'Muppet Treasure Island (1996)',
 '22': 'Braveheart (1995)',
 '23': 'Taxi Driver (1976)',
 '24': 'Rumble in the Bronx (1995)',
 '25': 'Birdcage, The (1996)',
 '26': 'Brothers McMullen, The (1995)',
 '27': 'Bad Boys (1995)',
 '28': 'Apollo 13 (1995)',
 '29': 'Batman Forever (1995)',
 '30': 'Belle de jour (1967)',
 '31': 'Crimson Tide (1995)',
 '32': 'Crumb (1994)',
 '33': 'Desperado (1995)',
 '34': 'Doom Genera

In [ ]:

!python3 main2.py

In [4]:
record

{'_id': {'$oid': '56acf43a135998708ec5df3e'},
 'genre': ['12'],
 'genre_code': '0000000000001000000',
 'id': 1683,
 'imdb_url': '发射的发射的发射的',
 'release_time': '2016-01-01',
 'title': '发射的费收费啥'}

In [3]:
ls ./static/pictures

Amistad (1997).jpg
Batman (1989).jpg
Boogie Nights (1997).jpg
Chasing Amy (1997).jpg
Die Hard: With a Vengeance (1995).jpg
Emma (1996).jpg
Everyone Says I Love You (1996).jpg
Get Shorty (1995).jpg
Get Shorty.jpg
Highlander (1986).jpg
Jerry Maguire (1996).jpg
Mission: Impossible (1996).jpg
Mother (1996).jpg
Mrs. Brown (Her Majesty, Mrs. Brown) (1997).jpg
Seven Years in Tibet (1997).jpg
Speed (1994).jpg
Star Trek: First Contact (1996).jpg
Trainspotting (1996).jpg
True Lies (1994).jpg
Ulee's Gold (1997).jpg
titanic.jpg
venom.jpg


In [2]:
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [3]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [ ]:
rec_movie_name

In [6]:
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [7]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [9]:
with open('show_movie_url_pic.pickle', 'rb') as f:
        show_movie_data = pickle.load(f)

In [11]:
show_movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '